# 데이터 설명

데이터 전처리용 데이터는 21~23년 신청현황 데이터 셋에 대한 전처리를 진행함

# 패키지 로드

In [5]:
import pandas as pd
import numpy as np
import os
import re
import warnings
warnings.filterwarnings('ignore')

In [6]:
file_name=os.listdir()[3]
file_name

'1) `21~23년 신청현황 (지역, 인원, 작물종류, 면적, 배정인원)1.xlsx'

# 1. 신청현황 데이터 불러오기

In [7]:
data=pd.read_excel(file_name,skiprows=2)

# 2. 중복 컬럼명 변경

- 지자체명=>지자체명_시도(시도 단위 컬럼)
- 지자체명=>지자체명_시군구(시군구 단위 컬럼)

In [4]:
new_column={'지자체명':'지자체명_시도','지자체명.1':'지자체명_시군구'}
data.rename(columns=new_column,inplace=True)
data.head(5)

,비고,지자체명_시도,지자체명_시군구,구분,"주소지 (조합, 법인 소재지)",농업경영체,배정신청 인원,지자체추가배정인원,합계,작물 종류,총 재배면적\n(단위 : 1000㎡),이력 여부,배정 인원,Unnamed: 13
0,2023,충청남도,홍성군,농가,충청남도 홍성군,1-222-426-920,3,NaN,NaN,⑧기타식량작물,139,없음,0,NaN
1,2023,충청남도,홍성군,농가,충청남도 홍성군,1-000-426-965,4,NaN,NaN,⑧기타식량작물,18,있음,3,NaN
2,2023,충청남도,홍성군,농가,충청남도 홍성군,1-000-413-619,5,NaN,NaN,⑧기타식량작물,18,있음,2,NaN
3,2023,충청남도,홍성군,법인,충청남도 홍성군,2-000-571-628,8,NaN,NaN,⑧기타식량작물,17,없음,0,NaN
4,2023,충청남도,홍성군,농가,충청남도 홍성군 광천읍 홍남동로126번길 150,1-000-858-146,5,NaN,NaN,⑦곡물,16,없음,0,NaN


# 3. 표기 오류 정제 & 공백삭제

## - 담양->담양군
## - 충청북도 청양군->충청남도 청양군

In [5]:
data['지자체명_시군구']=np.where(data['지자체명_시군구']=='담양','담양군',data['지자체명_시군구'])
data['지자체명_시도']=np.where((data['지자체명_시군구']=='청양군') & (data['지자체명_시도']=='충청북도'),'충청남도',data['지자체명_시도'])
data['지자체명_시도']=data['지자체명_시도'].str.strip()
data['지자체명_시군구']=data['지자체명_시군구'].str.strip()

In [6]:
data['지자체명_시도'].value_counts().sort_values() #개수 일치

세종특별자치시      28
제주특별자치도     313
경기도         519
경상남도        986
충청남도       1510
전라남도       1533
전라북도       1536
충청북도       1592
강원도        3140
경상북도       3208
Name: 지자체명_시도, dtype: int64

# 4. 농업경영체 컬럼 정제

# 4.1 특수문자 제거

In [7]:
data['농업경영체']=data['농업경영체'].astype(str) #문자형으로 일괄변경
print(data['농업경영체'].nunique()) #unique 일치

10821


In [8]:
#숫자 제외 모든 문자 제거
def extract_numbers(text):
    return re.sub(r'\D', '', text)

data['농업경영체']=data['농업경영체'].apply(extract_numbers)

In [9]:
data['농업경영체']=data['농업경영체'].apply(lambda x: re.sub('-', '', x)) #-제거
data['농업경영체']=data['농업경영체'].apply(lambda x: re.sub('  ', '', x))  # 특수문자
data['농업경영체']=data['농업경영체'].apply(lambda x: re.sub(' ', '', x))  # 특수문자
data['농업경영체']=data['농업경영체'].apply(lambda x: re.sub('\\*', '', x))  # *제거
data['농업경영체']=data['농업경영체'].apply(lambda x: re.sub(' ', '', x))  # 공백
data['농업경영체']=data['농업경영체'].apply(lambda x: re.sub('\xa0', '', x))  #특수문자제거
data['농업경영체']=data['농업경영체'].apply(lambda x: re.sub('\n', '', x))  # 줄바꿈
#print(data['농업경영체'].nunique()) #개수 일치

In [10]:
print(data['농업경영체'].nunique()) #개수 일치

10231


In [11]:
# 농업경영체 번호 각 자리수 계산
data['농업경영체_자리수']=data['농업경영체'].apply(lambda x:len(x))
data['농업경영체_자리수'].value_counts().sort_index()

0        35
8         2
9       197
10    14113
11       17
20        1
Name: 농업경영체_자리수, dtype: int64

# 4.2 자리수 확인 후 추가 정제 작업 진행

In [12]:
#삭제할 데이터
data=data[data['농업경영체_자리수']!=0] #자리수=0인 행 제거 
data=data[data['농업경영체_자리수']!=8] #8자리수 모두삭제

In [13]:
data[data['농업경영체_자리수']==9] #R에서 정제값과 같음
index_values=data.index #데이터 프레임 인덱스 추출

# 4.3 자리수 9 전처리

## 4.3.1 0으로 시작하는 농업경영체 1 붙여주기

In [14]:
for i in index_values:
    if (data.loc[i,'농업경영체_자리수']==9) and (data.loc[i,'농업경영체'][0]=='0'): #자리수가 9이고, 첫번째 자리수가 0인 행들 고르기
        data.loc[i,'농업경영체']='1'+data.loc[i,'농업경영체'] #해당 열의 농업경영체 앞자리에 1을 추가
    else:
        pass

## 4.3.2 자리수 10 전처리

In [15]:
data_10=data[data['농업경영체_자리수']==10] #R에서 정제값과 같음
index_values=data_10.index #데이터 프레임 인덱스 추출

In [16]:
del_list=[]
for i in index_values:
    if (data.loc[i,'농업경영체'][0]=='1') or (data.loc[i,'농업경영체'][0]=='2') or (data.loc[i,'농업경영체'][0]=='3'):
        pass
    else:
        del_list.append(i)

In [17]:
data.loc[del_list] #0으로 시작하거나, 공공형 계절 사업자와 같은 농업경영체 확인
data.drop(del_list,inplace=True)

# 4.3.3 10자리 이상 11자리부터 10자리만 남기고 모두 제거하기

In [18]:
# 농업경영체 번호 각 자리수 계산
data['농업경영체_자리수']=data['농업경영체'].apply(lambda x:len(x))
data['농업경영체_자리수'].value_counts().sort_index()

9        14
10    14294
11       17
20        1
Name: 농업경영체_자리수, dtype: int64

In [19]:
index_values1=data.query('농업경영체_자리수>10').index

In [20]:
for i in index_values1:
    if data.loc[i,'농업경영체'][0].isdigit(): #맨앞자리가 숫자인 경우만 10자리로 잘라주기
        data.loc[i,'농업경영체']=data.loc[i,'농업경영체'][0:10] #10자리수 만 가져오기
    else:
        pass

data['농업경영체_자리수']=data['농업경영체'].apply(lambda x:len(x)) #자리수 다시 계산
data['농업경영체_자리수'].value_counts().sort_index() #자리수 오름차순 정렬하여 빈도수 확인

9        14
10    14312
Name: 농업경영체_자리수, dtype: int64

In [21]:
final_data=data[data['농업경영체_자리수']==10] #자리수=10인 행으로 데이터 재구성 

# 5. 불필요컬럼 삭제 및 주요 변수 데이터 타입 변경

In [22]:
final_data['농업경영체']=pd.to_numeric(final_data['농업경영체'])
final_data.drop(columns=['농업경영체_자리수'],inplace=True) #농업경영체 자리수 삭제
final_data.drop(columns=['Unnamed: 13'],inplace=True) #14번째 컬럼 삭제

In [23]:
final_data['농업경영체'].nunique()

10154

In [27]:
len(final_data)

14312

# 6. 배정신청인원 컬럼 정제

In [28]:
final_data['배정신청 인원']=final_data['배정신청 인원'].astype(str)
final_data['배정신청 인원'].value_counts()

2     4678
5     1932
4     1870
1     1839
3     1296
0      962
6      767
9      527
7      240
8      194
10       6
12       1
Name: 배정신청 인원, dtype: int64

In [29]:
final_data['배정신청 인원']=final_data['배정신청 인원'].apply(lambda x: re.sub('-', '0', x))
final_data['배정신청 인원']=final_data['배정신청 인원'].apply(lambda x: re.sub('nan', '0', x))
final_data['배정신청 인원']=pd.to_numeric(final_data['배정신청 인원'])

In [30]:
len(final_data['배정신청 인원'].unique())

12

# 7. 지자체추가배정인원 컬럼 정제

In [31]:
final_data['지자체추가배정인원']=final_data['지자체추가배정인원'].astype(str)
final_data['지자체추가배정인원'].value_counts()

0      6914
nan    6552
-       483
1       163
2        83
없음       43
3        41
4        15
5         6
6         4
-2        4
-1        2
9         1
-4        1
Name: 지자체추가배정인원, dtype: int64

In [32]:
final_idx=final_data.index
for i in final_idx:
    if final_data.loc[i,'지자체추가배정인원']=='-':
        final_data.loc[i,'지자체추가배정인원']='0'
    else:
        pass

In [33]:
final_data['지자체추가배정인원']=final_data['지자체추가배정인원'].apply(lambda x: re.sub('nan', '0', x))
final_data['지자체추가배정인원']=final_data['지자체추가배정인원'].apply(lambda x: re.sub('없음', '0', x))
final_data['지자체추가배정인원']=pd.to_numeric(final_data['지자체추가배정인원'])

In [34]:
final_data['지자체추가배정인원'].value_counts()

 0    13992
 1      163
 2       83
 3       41
 4       15
 5        6
 6        4
-2        4
-1        2
 9        1
-4        1
Name: 지자체추가배정인원, dtype: int64

In [39]:
final_data['배정신청 인원'].value_counts()

2     4678
5     1932
4     1870
1     1839
3     1296
0      962
6      767
9      527
7      240
8      194
10       6
12       1
Name: 배정신청 인원, dtype: int64

# 8. 합계 컬럼 변경

In [40]:
final_data['합계']=final_data['배정신청 인원'] + final_data['지자체추가배정인원']

In [41]:
final_data['합계'].value_counts().sort_values('index')

13       2
11       7
12       8
10      25
8      204
7      256
9      524
6      808
0      929
3     1303
1     1827
4     1876
5     1890
2     4653
Name: 합계, dtype: int64

# 9. 작물 종류 컬럼 정제

In [42]:
final_index=final_data.index

In [43]:
final_data['작물 종류']=final_data['작물 종류'].astype(str)

In [44]:
for i in final_index:
    crop=final_data.loc[i,'작물 종류']
    if ('①' in crop) or ('1' in crop):
        final_data.loc[i,'작물 종류']='① 시설원예·특작'
        #시설원예 특작 수정
        
    elif ('②' in crop) or ('2' in crop):
        final_data.loc[i,'작물 종류']='② 버섯'
        #버섯 수정
        
    elif ('③' in crop) or ('3' in crop):
        final_data.loc[i,'작물 종류']='③ 과수'
        #과수 수정
        
    elif ('④' in crop) or ('4' in crop) or ('인삼' in crop):
        final_data.loc[i,'작물 종류']='④ 인삼, 일반채소'
        #인삼 일반채소 수정
        
    elif '⑤' in crop:
        final_data.loc[i,'작물 종류']='⑤ 종묘재배'
        #종묘재배 수정
        
    elif ('⑥' in crop) or ('기타원예' in crop):
        final_data.loc[i,'작물 종류']='⑥ 기타원예·특작'
        #기타원예 특작 수정
        
    elif ('⑦' in crop) or ('곡물' in crop):
        final_data.loc[i,'작물 종류']='⑦ 곡물'
        #곡물 수정
        
    elif ('⑧' in crop) or ('식량' in crop):
        final_data.loc[i,'작물 종류']='⑧ 기타 식량작물'
        # 식량 작물 수정
        
    elif '⑨' in crop:
        final_data.loc[i,'작물 종류']='⑨ 곶감가공'
        #곶감 수정
        
    else:
        pass

In [45]:
final_data['작물 종류'].value_counts()

① 시설원예·특작     6286
③ 과수          2929
④ 인삼, 일반채소    2330
⑦ 곡물           952
⑥ 기타원예·특작      690
⑧ 기타 식량작물      583
nan            297
⑤ 종묘재배         124
② 버섯           117
⑨ 곶감가공           4
Name: 작물 종류, dtype: int64

In [46]:
final_data.to_csv("1) 21~23년 신청현황 (지역, 인원, 작물종류, 면적, 배정인원)_ver2.csv", index = False, encoding='EUC-KR')